# Домашнее творческое задание

## 1. Парсинг (avito.ru)

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np

In [58]:
def get_html(url):
    r = requests.get(url)
    return r.text

def get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    divs = soup.find('div', class_ = 'pagination-pages clearfix')
    pages = divs.find_all('a', class_ = 'pagination-page')[-1].get('href')
    total_pages = pages.split('=')[1].split('&')[0]
    return int(total_pages)

def write_csv(data):
    with open('avito.csv', 'a', encoding = 'Utf-8') as f:
        writer = csv.writer(f)
        writer.writerow((data['title'],
                         data['qkomnat'],
                         data['meters'],
                         data['floor'],
                         data['floorsintotal'],
                         data['metro'],
                         data['metrokm'],
                         data['price']))
        
def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    divs = soup.find('div', class_ = 'js-catalog_serp')
    ads = divs.find_all('div', class_ = 'item_table-wrapper')
    for ad in ads:
        try:
            div = ad.find('div', class_ = 'description').find('h3')
            title = div.text.strip()
            title = title[:title.find(',')]
            if title == 'Студия':
                title
            else:
                title = title.split(' ')[1].replace('к', 'К')
        except:
            title = ''
        try:
            div = ad.find('div', class_ = 'description').find('h3')
            qkomnat = div.text
            qkomnat = int(qkomnat.split('-')[0])
        except:
            qkomnat = 1 
        try:
            div = ad.find('div', class_ = 'description').find('h3')
            meters = div.text.strip()
            meters = meters[meters.find(',') + 2 : meters.rfind('м') - 1]
        except:
            meters = ''
        try:
            div = ad.find('div', class_ = 'description').find('h3')
            floor = div.text.strip()
            floor = int(floor[floor.rfind(',') + 2 : floor.rfind('/')])
        except:
            floor = ''
        try:
            div = ad.find('div', class_ = 'description').find('h3')
            floorsintotal = div.text.strip()
            floorsintotal = int(floorsintotal[floorsintotal.rfind('/') + 1 : floorsintotal.rfind('эт.') - 1])
        except:
            floorsintotal = ''
        try:
            div = ad.find('div', class_ = 'item-address')
            metro = div.text.strip()
            metro = metro[:metro.find(',')]
        except:
            metro = ''
        try:
            div = ad.find('div', class_ = 'item-address')
            metrokm = div.text
            metrokm = metrokm.replace('\t', '').replace(', ', ',\r\n').split('\n')[2][:-1]
        except:
            metrokm = ''
        try:
            price = ad.find('div', class_ = 'about').text.strip()
            price = int(price[:price.find('₽') - 1].replace(' ', ''))
        except:
            price = ''
        data = {'title': title,
                'qkomnat': qkomnat,
                'meters': meters,
                'floor': floor,
                'floorsintotal': floorsintotal,
                'metro': metro,
                'metrokm': metrokm,
                'price': price}
        write_csv(data)
        
def main():
    url = 'https://www.avito.ru/moskva/kvartiry/prodam?metro=10-13-48-64-66-73-99-104-117-137&p=1'
    base_url = 'https://www.avito.ru/moskva/kvartiry/prodam?'
    query_par = 'metro=10-13-48-64-66-73-99-104-117-137&'
    page_part = 'p='
    #total_pages = get_total_pages(get_html(url))
    for i in range(0, 22):
        url_gen = base_url + query_par + page_part + str(i)
        html = get_html(url_gen)
        get_page_data(html)
if __name__ == '__main__':
    main()

In [2]:
avito = pd.read_csv(r'avito.csv', sep = ',', header = None)
avito.columns = ['Title', 'Rooms', 'Square', 'Floor', 'FloorsInTotal', 'Metro', 'DistanceFromMetro', 'Price']
avito.head(10)

,Title,Rooms,Square,Floor,FloorsInTotal,Metro,DistanceFromMetro,Price
0,Квартира,2,79.2,6,13,Маяковская,300 м,50000000
1,Квартира,1,345.1,8,9,Белорусская,900 м,185772820
2,Квартира,5,229.0,5,5,Баррикадная,500 м,99700000
3,Квартира,2,87.1,13,18,Белорусская,1 км,51710399
4,Квартира,3,54.7,2,8,Савеловская,300 м,11400000
5,Квартира,2,47.0,5,7,Маяковская,300 м,11700000
6,Квартира,3,115.3,7,24,Белорусская,800 м,44277270
7,Квартира,3,130.4,15,24,Белорусская,800 м,54791793
8,Квартира,2,44.0,1,5,Краснопресненская,500 м,10300000
9,Студия,1,19.4,1,5,Маяковская,500 м,5990000
